In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
data_set = pd.read_csv('./WikiQA-train.tsv', sep="\t")
data_set.head()

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label
0,Q1,how are glacier caves formed?,D1,Glacier cave,D1-0,A partly submerged glacier cave on Perito More...,0
1,Q1,how are glacier caves formed?,D1,Glacier cave,D1-1,The ice facade is approximately 60 m high,0
2,Q1,how are glacier caves formed?,D1,Glacier cave,D1-2,Ice formations in the Titlis glacier cave,0
3,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1
4,Q1,how are glacier caves formed?,D1,Glacier cave,D1-4,"Glacier caves are often called ice caves , but...",0


In [3]:
questions = data_set['Question'].unique().tolist()
#print((questions))
answers = data_set['Sentence'].unique().tolist()
# question = 1
# answer = 0
train_X_1 = pd.DataFrame(data=questions,columns=['sentences'])
train_Y_1 = pd.DataFrame(data=[1 for _ in range(len(questions))], columns=['label'])

train_X_2 = pd.DataFrame(data=answers[:len(questions)], columns=['sentences'])
train_Y_2 = pd.DataFrame(data=[0 for _ in range(len(questions))], columns=['label'])

train_X = pd.concat([train_X_1, train_X_2], axis=0)
train_Y = pd.concat([train_Y_1, train_Y_2], axis=0)

train_data = pd.concat([train_X, train_Y], axis=1)

train = train_data.sample(frac=1).reset_index(drop=True)
train.head()


,sentences,label
0,how many rounds in boxing,1
1,The powers not delegated to the United States ...,0
2,The overall reaction is broken into many small...,0
3,what school district is alvin in?,1
4,what is the book of verses james frey reads in...,1


In [4]:
# print(train['sentences'].tolist())

In [5]:
import spacy
import copy
import numpy as np
nlp = spacy.load('en_core_web_lg')  # make sure to use larger model!

In [6]:


max_size = 70
emb_dim = 300

def preprocess_data(data):
    sen, lab = data
    ip_vec = []
    idx_rm = []
    for idx,sentence in enumerate(sen):

        sentence = sentence.replace("?","")
        tokens = nlp(sentence)
        if(len(tokens)> max_size):
            idx_rm.append(idx)
            continue
        new_vec = []
        for token in tokens:
            if(not(token.text == '?' )):
                #print("emb dim =",len(token.vector))
                new_vec.append(token.vector)
        nc3 = [[0]*emb_dim]*(max_size-len(new_vec))
        new_vec.extend(nc3)
  
        ip_vec.append(copy.copy(new_vec))
    ip_vec = np.array(ip_vec)
    ip_labels = [lab[i] for i in range(len(lab)) if i not in idx_rm]
    return ip_vec, ip_labels


In [7]:
#print(train)
#print(train)
tdata,tlabel = preprocess_data((train['sentences'], train['label']))
print(len(tlabel))

tdata = np.reshape(tdata,(-1,max_size*emb_dim))


4223


In [8]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(tdata, tlabel)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=1e-05, verbose=0)

In [9]:
test = ["What was the network operated by the Duct PTT Telecom",
"When did Zhenjin die",
"The force, therefore, is related directly to the difference in potential energy between two different locations in space, and can be considered to be an artifact of the potential field in the same way that the direction and amount of a flow of water can be considered to be an artifact of the contour map of the elevation of an area",
"In 1785 what paper did James Hutton present to the Royal Society of Edinburgh",
"What does the ctenophora use to swim",
"What city was Jan van Riebeeck from",
"At the same time, the available evidence provides little support for the view that U.S.-style labor-market flexibility dramatically improves labor-market outcomes",
"Acute oxygen toxicity (causing seizures, its most feared effect for divers) can occur by breathing an air mixture with 21% O 2 at 66 m or more of depth; the same thing can occur by breathing 100% O 2 at only 6 m",
"What does poor academic achievement lead to according to critics",
"What does oxygen the basis for in combustion",
"Is this for real",
"How many bits are often in the primes used for RSA private key cryptography algorithms",
"What was lacking in the second assessment report",
"The graph isomorphism problem, the discrete logarithm problem and the integer factorization problem are examples of problems believed to be NP-intermediate",
"Late outbreaks in central Europe included the Italian Plague of 1629–1631, which is associated with troop movements during the Thirty Years' War, and the Great Plague of Vienna in 1679",
"Gasquet (1908) claimed that the Latin name atra mors (Black Death) for the 14th-century epidemic first appeared in modern times in 1631 in a book on Danish history by J.I",
"What kind of focus has the Ethical Reasoning program had since 2007",
"What has having an EU Citizenship decreased",
"Which group has two layers of cells with a middle layer of mesoglea",
"How far is it from Davenports Neck to New Rochelle",
"Much of the work of the Scottish Parliament is done in committee",
"Release of carbon contained in vegetation will slow down what",
"Following the death of Braddock, William Shirley assumed command of British forces in North America",
"What are the least commonly ascribed attributes of L in relation to P",
"Even before the Norman Conquest of England, the Normans had come into contact with Wales",
"About how many cubic meters of make-up water is used by a 700-megawatt coal-fired power plant for evaporative cooling hourly",
"What is one thing that copepods like to eat",
"The Central Region, consisting of present-day Hebei, Shandong, Shanxi, the south-eastern part of present-day Inner Mongolia and the Henan areas to the north of the Yellow River, was considered the most important region of the dynasty and directly governed by the Central Secretariat (or Zhongshu Sheng) at Khanbaliq (modern Beijing); similarly, another top-level administrative department called the Bureau of Buddhist and Tibetan Affairs (or Xuanzheng Yuan) held administrative rule over the whole of modern-day Tibet and a part of Sichuan, Qinghai and Kashmir"]
test_labels = [1,1,0,1,1,1,0,0,1,1,1,1,1,0,0,0,1,1,1,1,0,1,0,1,0,1,1,0]

In [10]:
test_vec, test_labels = preprocess_data((test, test_labels))
test_vec = np.reshape(test_vec,(-1,max_size*emb_dim))

In [11]:
preds = clf.predict(test_vec)

In [12]:
print(preds)

[1 1 0 0 1 1 0 0 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 1]


In [13]:
count = 0
for i in range(len(test_labels)):
    if(test_labels[i]== preds[i]):
        count += 1
acc = count * 100.0/len(test_labels)
print(acc)

85.18518518518519


In [25]:
test_data = []
with open('./test-inputs.txt', 'r') as f:
    for line in f:
        test_data.append(line.strip())
f.close()

In [26]:
test_data[:10]

['What was the network operated by the Duct PTT Telecom',
 'When did Zhenjin die',
 'The force, therefore, is related directly to the difference in potential energy between two different locations in space, and can be considered to be an artifact of the potential field in the same way that the direction and amount of a flow of water can be considered to be an artifact of the contour map of the elevation of an area',
 'In 1785 James Hutton presented what paper to the Royal Society of Edinburgh',
 'What does the ctenophora use to swim',
 'It is the county seat of Duval County, with which the city government consolidated in 1968',
 'Where is the Asian gold miners strongest in Victoria',
 'How did france differ from Britain in managing its colonies',
 'In case of a shared physical medium (such as radio or 10BASE5), the packets may be delivered according to a multiple access scheme',
 'To clear out this confusion about the definition of imperialism one could speak of "formal" and "informal"

In [30]:
test_vec_1, _ = preprocess_data((test_data, test_labels))

In [32]:
test_vec_1 = np.reshape(test_vec_1,(-1,max_size*emb_dim))

In [34]:
preds_1 = clf.predict(test_vec_1)

In [36]:
file = open('svc_output.txt', 'w')
for val in preds_1:
    file.write(str(val)+"\n")
file.close()